# Model quality check and improvement workflow

1.Download the initila model constructed from ModelSEED. The format of initial model is SBML file.


2.generate an excel file for the model for easy checking. 

In [1]:
#2.1 excel format model generation. for checking and revising model, such as reactions and their lower or upper bound information.
import sbmltoexcel
sbmltoexcel.sbml_excel('Seed1111.5.65232.xml','Seed1111.5.65232.xlsx')# sbml to excel
#sbmltoexcel.model_excel(model_1,'model_1.xlsx') # cobrapy model to excel. If we want to check the model we can change it to excel formate at any time

In [ ]:
#2.2 change the excel format to sbml format.
import exceltosbml
exceltosbml.excel_sbml('Seed1111.5.65232.xlsx')

3 initial check and model revision

In [2]:
#3.1 import some modules and read the sbml model
from __future__ import print_function
import cobra
import re
import pandas as pd
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import pfba
model = cobra.io.read_sbml_model('Seed1111.5.65232.xml') # read sbml format model

In [7]:
#3.2 Calculate optimal growth rate and check its input and output
model.optimize()
model.summary() #check input/outputs for the optimal solution
#model.medium
# many substrates can be consumed, leading to unreasonable high growth rates, need to modify the boundaries of exchange reactions

IN FLUXES             OUT FLUXES            OBJECTIVES
--------------------  --------------------  -------------
cpd00162_e0    1e+03  cpd00001_e0    1e+03  biomass0  149
cpd00794_e0    1e+03  cpd00011_e0    1e+03
cpd00106_e0  692      cpd00013_e0    1e+03
cpd00588_e0  358      cpd00027_e0    1e+03
cpd00307_e0  226      cpd00067_e0    1e+03
cpd00007_e0  196      cpd00221_e0    1e+03
cpd00075_e0  178      cpd00142_e0  811
cpd00009_e0  121      cpd00036_e0  352
cpd00129_e0  104      cpd00100_e0  215
cpd00276_e0   68.7    cpd00092_e0  195
cpd00039_e0   46.3    cpd00156_e0  171
cpd00226_e0   45.1    cpd00797_e0  157
cpd00051_e0   36.8    cpd00082_e0   13.5
cpd00107_e0   36.7
cpd00322_e0   36.1
cpd00105_e0   30.9
cpd11581_e0   29.9
cpd00066_e0   23
cpd11591_e0   19.5
cpd15604_e0   19.3
cpd15606_e0   18
cpd15603_e0   12.7
cpd01080_e0   12.7
cpd00119_e0   11.8
cpd00065_e0    7.04
cpd03847_e0    3.73
cpd10515_e0    1.85
cpd00393_e0    1.39
cpd11606_e0    0.924
cpd00034_e0    0.462
cpd00030_e0 

{'EX_cpd00001_e0': 1000.0,
 'EX_cpd00007_e0': 1000.0,
 'EX_cpd00009_e0': 1000.0,
 'EX_cpd00011_e0': 1000.0,
 'EX_cpd00012_e0': 1000.0,
 'EX_cpd00013_e0': 1000.0,
 'EX_cpd00023_e0': 1000.0,
 'EX_cpd00027_e0': 1000.0,
 'EX_cpd00030_e0': 1000.0,
 'EX_cpd00034_e0': 1000.0,
 'EX_cpd00036_e0': 1000.0,
 'EX_cpd00039_e0': 1000.0,
 'EX_cpd00041_e0': 1000.0,
 'EX_cpd00048_e0': 1000.0,
 'EX_cpd00051_e0': 1000.0,
 'EX_cpd00058_e0': 1000.0,
 'EX_cpd00060_e0': 1000.0,
 'EX_cpd00063_e0': 1000.0,
 'EX_cpd00064_e0': 1000.0,
 'EX_cpd00065_e0': 1000.0,
 'EX_cpd00066_e0': 1000.0,
 'EX_cpd00067_e0': 1000.0,
 'EX_cpd00069_e0': 1000.0,
 'EX_cpd00073_e0': 1000.0,
 'EX_cpd00075_e0': 1000.0,
 'EX_cpd00076_e0': 1000.0,
 'EX_cpd00082_e0': 1000.0,
 'EX_cpd00092_e0': 1000.0,
 'EX_cpd00098_e0': 1000.0,
 'EX_cpd00099_e0': 1000.0,
 'EX_cpd00100_e0': 1000.0,
 'EX_cpd00105_e0': 1000.0,
 'EX_cpd00106_e0': 1000.0,
 'EX_cpd00107_e0': 1000.0,
 'EX_cpd00118_e0': 1000.0,
 'EX_cpd00119_e0': 1000.0,
 'EX_cpd00122_e0': 1000.0,
 

In [5]:
#3.3 Change boundary of exchange reactions
'''
1. normally only allow one carbon source (default was glucose), uptake rate was set at 10mmol/g/h.
2. oxygen exchange rate can be set to 0 for anaerobic or unlimited for aerobic
3. CO2 exchange rate can be negative for CO2 fixation or 0 so that CO2 can only be released
4. nitrogen (NH3) sulfur (H2S2O3,sulfate) and phosphate (Pi) sources can also be set, normally not limited
5. exchange reactions for ions(Pb,Cd2,Cu2,Fe2,Fe3,Mg,K,Zn2,Na,Co2,Cl,Hg2,Mn2,Ca2) and H2O were unconstrained.
3. boundaries of all other exchange reactions were set to 0-1000 so that the metabolites can only be produced but not consumed. 
'''
c_exchange=['EX_cpd00027_e0'] #exchange reaction for carbon sources, use list to allow multiple sources
for ex in c_exchange:
    r=model.reactions.get_by_id(ex)
    r.bounds=(-10, 1000) # maximal uptake rate 10mmol/g/h
o_exchange=['EX_cpd00007_e0'] #exchange reaction for oxygen for anaerobic or aerobic conditions
for ex in o_exchange:
    r=model.reactions.get_by_id(ex)
    r.bounds=(-1000, 1000) 
co2_exchange=['EX_cpd00011_e0'] #exchange reaction for CO2, 
for ex in co2_exchange:
    r=model.reactions.get_by_id(ex)
    r.bounds=(0, 1000) 
n_exchange=['EX_cpd00013_e0'] #exchange reaction for N sources
for ex in n_exchange:
    r=model.reactions.get_by_id(ex)
    r.bounds=(-1000, 1000) 
s_exchange=['EX_cpd00048_e0','EX_cpd00268_e0'] #exchange reaction for sulafte and H2S2O3
for ex in s_exchange:
    r=model.reactions.get_by_id(ex)
    r.bounds=(-1000, 1000) 
p_exchange=['EX_cpd00009_e0','EX_cpd00012_e0'] #exchange reaction for Pi and PPi
for ex in p_exchange:
    r=model.reactions.get_by_id(ex)
    r.bounds=(-1000, 1000) 
ion_exchange=['EX_cpd04097_e0','EX_cpd00001_e0','EX_cpd00058_e0','EX_cpd10515_e0','EX_cpd10516_e0','EX_cpd00254_e0','EX_cpd00205_e0',
            'EX_cpd00034_e0','EX_cpd00971_e0','EX_cpd00149_e0','EX_cpd00099_e0','EX_cpd00531_e0','EX_cpd00030_e0','EX_cpd00063_e0']
preset_exchange=c_exchange+o_exchange+co2_exchange+n_exchange+s_exchange+p_exchange+ion_exchange
for rea in model.boundary: 
    if rea.id not in preset_exchange:
        rea.bounds=(0,1000)
model.optimize()
model.summary() 
cobra.io.write_sbml_model(model, "model01.xml")
#growth likely to be zero, gaps in the network, but first check ATP production

IN FLUXES       OUT FLUXES    OBJECTIVES
--------------  ------------  ------------
cpd00001_e0  0                biomass0  0
cpd00007_e0  0
cpd00009_e0  0
cpd00011_e0  0
cpd00012_e0  0
cpd00013_e0  0
cpd00023_e0  0
cpd00027_e0  0
cpd00030_e0  0
cpd00034_e0  0
cpd00036_e0  0
cpd00039_e0  0
cpd00041_e0  0
cpd00048_e0  0
cpd00051_e0  0
cpd00058_e0  0
cpd00060_e0  0
cpd00063_e0  0
cpd00064_e0  0
cpd00065_e0  0
cpd00066_e0  0
cpd00067_e0  0
cpd00069_e0  0
cpd00073_e0  0
cpd00075_e0  0
cpd00076_e0  0
cpd00082_e0  0
cpd00092_e0  0
cpd00098_e0  0
cpd00099_e0  0
cpd00100_e0  0
cpd00105_e0  0
cpd00106_e0  0
cpd00107_e0  0
cpd00118_e0  0
cpd00119_e0  0
cpd00122_e0  0
cpd00129_e0  0
cpd00130_e0  0
cpd00138_e0  0
cpd00142_e0  0
cpd00149_e0  0
cpd00156_e0  0
cpd00159_e0  0
cpd00162_e0  0
cpd00179_e0  0
cpd00205_e0  0
cpd00209_e0  0
cpd00211_e0  0
cpd00214_e0  0
cpd00221_e0  0
cpd00222_e0  0
cpd00226_e0  0
cpd00254_e0  0
cpd00264_e0  0
cpd00268_e0  0
cpd00276_e0  0
cpd00307_e0  0
cpd00308_e0  0
cpd0

4 Checking and revise the ATP production pathway 

In [6]:
#4.1 Check the wrong ATP generation cycle, using the ATP consumption reaction (maintenance) rxn00062 as objective reaction
#find the maintenance reaction by searching in excel, should change the lower bound to maintenace coefficient such as 4.9 
#rin=model.reactions.get_by_id("EX_cpd00027_e0") # check the glucose input
#rin.bounds=(0, 1000) # we can turn off the carbon input
atpm=model.reactions.get_by_id("rxn00062_c0")
atpm.bounds=(4.9, 1000) #maintenace rate as lower bound
#atpm.bounds=(-1000, 1000) #修改以使其可最小化净生成ATP，从而可计算净消耗ATP的无效循环
model.objective=atpm # change objective function
model.optimize()
model.summary() 
#unreasonable high atp production rate, need to check the pathway to find what is wrong

-10
IN FLUXES              OUT FLUXES             OBJECTIVES
---------------------  ---------------------  ------------------
cpd00007_e0  3.41e-13  cpd00001_e0  3.59e-13  rxn00062_c0  1e+03
cpd00013_e0  1.17e-13  cpd00011_e0  3.53e-13
cpd00067_e0  6.15e-14  cpd10516_e0  6.36e-15
cpd00027_e0  5.27e-14  cpd00393_e0  1.86e-17
                       cpd00048_e0  1.53e-17


In [7]:
#4.2 output the pathway to a text file to find the mistakes
import output
outputfile_name ='atp.txt' # provide output file name
fluxes = pfba(model).fluxes #use pfba to get a simple path
output.pathway(fluxes,outputfile_name)

rxn10043_c0 1000.0000000000002
rxn10042_c0 1000.0000000000003
rxn00058_c0 -500.0000000000001
rxn00062_c0 1000.0


By checking the result pathway file, we can find ATP without consumption of any substrate but from a cycle pathway. Reaction flux for rxn00058_c0 ((4.0)H_c0 + (4.0)Cytochrome_c2_c0 + O2_c0 -->(2.0)H2O_c0 + (4.0)Cytochrome_c3_c0) is -500, however this reaction	should be irreversible as O2_c0 should only be consumed but not produced in the respiratory chain. This should be corrected by change the lower bound of this reaction to zeor to make it irreversible. 
Actually most errors in ATP generation were caused by mistakes in reaction reversibility, we need to repeat the process untill all erros are corrected and a reasonable amount of ATP is produced from the carbon source.We can use of a set of rules to quickly determine which reaction in a wrong pathway should be set to irreversible.
1.	Oxygen consumption reactions,
2.	Most of the carbon dioxide production reaction (CO2 is only used as substrate when a high energy substrate such as PEP (phophoenolpyruvate) and ATP is consumed at the same time),
3.	Most of the NH3 production reaction (NH3 is only used in two NH3 assimilation reactions and carbamoyl phosphate production),
4.	Most of the phosphate production reaction (the reaction is regarded as reversible only when phosphate reacts with another high energy substrate such as AcCoA)
5.	Reactions in which S-Adenosyl-L-methionine is converted to S-Adenosyl-L-homocystine for providing a methyl group,
6.	Reactions in which tetrahydrofolate (THF) is produced for transferring one carbon unit, 
7.	Most of the ATP (or other high energy metabolites) consumption reactions, except for reactions with another high energy metabolite such as GTP, CAP, acetyl phosphate, Acyl-CoA,
8.	UDP-sugar consumption reactions for transferring sugar units,  
9.	CDP-diacylglycerol consumption reactions for phosphatidyl group transfer,
10.	reactions like: 3'-Phosphoadenylylsulfate (PAPS) + A = adenosine 3',5'-bisphosphate (PAP) + B

To avoid the time consuming correction process, we compiled a list of reactions from ModelSeed that should be corrected to irreversible to avoid wrong ATP generation pathways. The model can be updated from this list. 
请生成一个修正反应列表，文本文件含三列数据 反应ID，下界 上界，由此文件更新sbml文件的model02.


In [42]:
#4.3 According to calculated results of 4.2 revise model,then we will get 2.0 version
rea_need_forward = ['rxn05604_c0','rxn05297_c0','rxn05638_c0'] # boundary change to 0-1000
model_2=model_1
for i in rea_need_forward:
    model_2.reactions.get_by_id(i).bounds=(0, 1000)
    
rea_need_reversible = ['rxn05209_c0'] # boundary change to -1000-1000
for j in rea_need_reversible:
    model_2.reactions.get_by_id(j).bounds=(-1000, 0)

# After correcting, repeat 4.2 step

查找净消耗ATP的无效循环（模块计算顺序为1,2,3,4,7）

这样的循环会很多，若查找经过某特定反应的循环，可将该反应设为一较大值，如100（模块计算顺序为1,2,3,4,6,7）

In [ ]:
#4 无效循环目标设置
model.objective_direction='min'

查看是否有NAD（P）H无限生成或消耗反应，以及将NADPH转化为NADH的无效循环

In [6]:
#5 还原力平衡分析
model.objective=model.reactions.get_by_id("NAD")

In [105]:
#3.4 Calculate the biomass precursors to check which matabolites can not be generated
import makefolder

outfolder = 'biomass_precursors'
makefolder.mkdir (outfolder)


biomass_precursors 创建成功


True

In [ ]:
#要修改的输入输出代谢物列表,计算同时产生多个结果文件
substrates=['succ_e']
products=['trp__L_c','arg__L_c']#带e下标代谢物不一定存在

In [42]:
for s in substrates:
    for p in products:
        newmodel=model.copy()
        newsub=newmodel.metabolites.get_by_id(s)
        newmodel.add_boundary(newsub, type='sink',ub=10.0)
        newpro=newmodel.metabolites.get_by_id(p)
        newmodel.objective =newmodel.add_boundary(newpro, type='demand') 
        outputfile=s+'_'+p+'.txt'
        flux = open(outputfile, 'w')
        fluxes = pfba(newmodel).fluxes
        for r,v in fluxes.iteritems():
            if abs(v)>1e-6:
                flux.write(r+'\t'+newmodel.reactions.get_by_id(r).build_reaction_string()+'\t'+str(v)+str(abs(v))+'\n')
        flux.close()
        #newmodel.summary() #模型输入输出通量，检查是否有不合理输入输出
        #newmodel.metabolites.nh4_c.summary() #检查某一代谢物相关通量，如氨主要通过哪些反应进入

NameError: name 'substrates' is not defined